# Selection network architecture with genetics algorithm

This notebook is inspired by the following paper: [Efficient Architecture Search for Deep Neural Networks](https://www.sciencedirect.com/science/article/pii/S1877050920303859)

In [87]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt
import numpy as np
import copy


### Import dataset

In [163]:
from script import cifar10
trainloader, testloader, classes = cifar10.cifar10()

Files already downloaded and verified
Files already downloaded and verified


### Define basic network

In [ ]:
class _Transition(nn.Sequential):
    def __init__(self, num_input_features, num_output_features):
        super(_Transition, self).__init__()
        self.add_module('BN', nn.BatchNorm2d(num_input_features))
        self.add_module('1x1', nn.Conv2d(num_input_features, num_output_features,
                                          kernel_size=1, stride=1, bias=False))
        self.add_module('relu', nn.ReLU(inplace=True))
        self.add_module('pool', nn.AvgPool2d(kernel_size=2, stride=2))


#define dense layer and bottleneck layer
class _DenseLayer(nn.Module):
    def __init__(self, num_input_features, growth_rate, bn_size, drop_rate, memory_efficient=False):
        super(_DenseLayer, self).__init__()
        self.add_module('BN_1', nn.BatchNorm2d(num_input_features))
        self.add_module('conv1', nn.Conv2d(num_input_features, bn_size *
                                           growth_rate, kernel_size=1, stride=1,
                                           bias=False))
        self.add_module('relu_1', nn.ReLU(inplace=True))
        self.add_module('BN_2', nn.BatchNorm2d(bn_size * growth_rate))
        self.add_module('conv3', nn.Conv2d(bn_size * growth_rate, growth_rate,
                                           kernel_size=3, stride=1, padding=1,
                                           bias=False)) # questo è il cromosoma
        self.add_module('relu_2', nn.ReLU(inplace=True))
        self.drop_rate = float(drop_rate)
        self.memory_efficient = memory_efficient

    def bn_function(self, inputs):
        "Bottleneck function"
        # type: (List[Tensor]) -> Tensor
        concated_features = torch.cat(inputs, 1)
        bottleneck_output = self.relu_1(self.conv1(self.BN_1(concated_features)))  # noqa: T484
        return bottleneck_output

    def forward(self, input):  # noqa: F811
        if torch.is_tensor(input):
            prev_features = [input]
        else:
            prev_features = input

        bottleneck_output = self.bn_function(prev_features)
        new_features = self.relu_2(self.conv3(self.BN_2(bottleneck_output)))
        if self.drop_rate > 0:
            new_features = F.dropout(new_features, p=self.drop_rate,
                                     training=self.training)
        return new_features



class _DenseBlock(nn.ModuleDict):
    _version = 2

    def __init__(self, num_layers, num_input_features, bn_size, growth_rate, drop_rate, memory_efficient=False):
        super(_DenseBlock, self).__init__()
        for i in range(num_layers):
            layer = _DenseLayer(
                num_input_features + i * growth_rate,
                growth_rate=growth_rate,
                bn_size=bn_size,
                drop_rate=drop_rate,
                memory_efficient=memory_efficient,
            )
            self.add_module('denselayer%d' % (i + 1), layer)

    def forward(self, init_features):
        features = [init_features]
        for name, layer in self.items():
            new_features = layer(features)
            features.append(new_features)
        return torch.cat(features, 1)

class DenseNet(nn.Module):
    def __init__(self, growth_rate=32, block_config=(12, 12, 12, 12),
                 num_init_features=64, bn_size=4, drop_rate=0, num_classes=10, memory_efficient=False):

        super(DenseNet, self).__init__()

        #Convolution and pooling part from table-1
        self.features = nn.Sequential(OrderedDict([
            ('conv0', nn.Conv2d(3, num_init_features, kernel_size=7, stride=2,
                                padding=3, bias=False)),
            ('norm0', nn.BatchNorm2d(num_init_features)),
            ('relu0', nn.ReLU(inplace=True)),
            ('pool0', nn.MaxPool2d(kernel_size=3, stride=2, padding=1)),
        ]))
       # self.features = nn.Sequential()
        # Add multiple denseblocks based on config 
        # for densenet-121 config: [6,12,24,16]
        num_features = num_init_features
        for i, num_layers in enumerate(block_config):
            block = _DenseBlock(
                num_layers=num_layers,
                num_input_features=num_features,
                bn_size=bn_size,
                growth_rate=growth_rate,
                drop_rate=drop_rate,
                memory_efficient=memory_efficient
            )
            self.features.add_module('denseblock%d' % (i + 1), block)
            num_features = num_features + num_layers * growth_rate
            if i != len(block_config) - 1:
                # add transition layer between denseblocks to 
                # downsample
                trans = _Transition(num_input_features=num_features,
                                    num_output_features=num_features // 2)
                self.features.add_module('transition%d' % (i + 1), trans)
                num_features = num_features // 2

        # Final batch norm
        self.features.add_module('norm5', nn.BatchNorm2d(num_features))

        # Linear layer
        self.classifier = nn.Linear(num_features, num_classes)

        # Official init from torch repo.
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight)
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.Linear):
                nn.init.constant_(m.bias, 0)

    def forward(self, x):
        features = self.features(x)
        out = F.relu(features, inplace=True)
        out = F.adaptive_avg_pool2d(out, (1, 1))
        out = torch.flatten(out, 1)
        out = self.classifier(out)
        return out

net = DenseNet()

### Define chromosomes

In this version each image will feed in the whole network with different kernel size. The kernel size will be selected by the genetic algorithm.

In [149]:
chromo = [ [nn.Conv2d(3,3,kernel_size=(1,3) ), nn.ReLU(), nn.BatchNorm2d(3), nn.Conv2d(3,3,kernel_size=(3,1))], [nn.Conv2d(3,3,kernel_size=(1,7)),nn.ReLU(), nn.BatchNorm2d(3), nn.Conv2d(3,3,kernel_size=(7,1))], 
[nn.Conv2d(3,3,3)], [nn.Conv2d(3,3,3, dilation=2)], [nn.Conv2d(3,3,1)] ]   # mancano le depthwise convolution 

Create an instance of the network with the corresponding chromosome:

In [154]:
copia = copy.deepcopy(chromo)
print(copia[0:2])
individual = Individual(copia[0:2])

[[Conv2d(3, 3, kernel_size=(1, 3), stride=(1, 1)), ReLU(), BatchNorm2d(3, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True), Conv2d(3, 3, kernel_size=(3, 1), stride=(1, 1))], [Conv2d(3, 3, kernel_size=(1, 7), stride=(1, 1)), ReLU(), BatchNorm2d(3, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True), Conv2d(3, 3, kernel_size=(7, 1), stride=(1, 1))]]
BatchNorm2d(3, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
Conv2d(3, 3, kernel_size=(1, 1), stride=(1, 1))
ReLU()
AvgPool2d(kernel_size=(2, 2), stride=(2, 2), padding=0)
BatchNorm2d(3, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
Conv2d(3, 3, kernel_size=(1, 3), stride=(1, 1))
ReLU()
BatchNorm2d(3, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
Conv2d(3, 3, kernel_size=(3, 1), stride=(1, 1))
ReLU()
BatchNorm2d(3, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
Conv2d(3, 3, kernel_size=(1, 1), stride=(1, 1))
ReLU()
AvgPool2d(kernel_size=(2, 2), s

In [110]:
print(copia[0:3])

[[Conv2d(3, 3, kernel_size=(3, 3), stride=(1, 1)), Conv2d(3, 3, kernel_size=(3, 1), stride=(1, 1)), BatchNorm2d(3, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True), Conv2d(3, 3, kernel_size=(1, 1), stride=(1, 1)), ReLU(), MaxPool2d(kernel_size=(2, 2), stride=(2, 2), padding=0, dilation=1, ceil_mode=False)], [Conv2d(3, 3, kernel_size=(1, 7), stride=(1, 1)), Conv2d(3, 3, kernel_size=(7, 1), stride=(1, 1)), BatchNorm2d(3, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True), Conv2d(3, 3, kernel_size=(1, 1), stride=(1, 1)), ReLU(), MaxPool2d(kernel_size=(2, 2), stride=(2, 2), padding=0, dilation=1, ceil_mode=False)]]


Plot the the output of the network: